In [ ]:
# 시계열 데이터 - 

시계열 데이터 

1.시간정보가 들어있는 데이터
*   "세상에는 시계열 데이터가 참 많다
*   단어가 나타내는 순서가 중요하다
*   샘플의 길이가 다름    
2.시계열 데이터를 인식하는 고전적인 모델
*   ARIMA, SARIMA
*   Prophet 등...
3.시계열 데이터를 인식하는 딥러닝 모델
*   순환 신경망(RNN)
*   LSTM(Long short term memory) : 선별기억능력을 갖춰 장기 문맥처리에 유리


매일 기온,  습도, 미세먼지 농도를 기록한다.  a1 = (23.2,42,0.25) a2=(, , ,)
* x = (a1,a2,a3 ,,,, at)
* 대표적인 응용은 미래 예측(prediction 또는 forecasting)
1.   내일 주가예측
2.   내일 날씨 예측
3.   기계의 고장 예측
4.   풍속과 풍향 예측(풍력 발전기의 효율 향상)
5.   농산물 가격/수요량 예측
6.   언어번역에 응용
7.   생성모델에 응용(사진을 보고 설명 문장 생성)


비트코인 예측하기

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [99]:
f = open('bitcoin.csv','r')
df = pd.read_csv(f,header=0)


Closing Price (USD) 종가만 가져온다

In [100]:
seq = df[['Closing Price (USD)']].to_numpy()

In [103]:
def seq2dataset(seq,window,horizon):
    X=[]; Y=[]
    for i in range(len(seq)-(window+horizon)+1):
        x=seq[i:(i+window)]
        y=(seq[i+window+horizon-1])
        X.append(x); Y.append(y)
    return np.array(X), np.array(Y)

In [104]:
w=7 # 윈도우 크기
h=1 # 수평선 계수

X,Y = seq2dataset(seq,w,h)

In [105]:
# 훈련 집합과 테스트 집합으로 분할
split=int(len(X)*0.7)
x_train=X[0:split]; y_train=Y[0:split]
x_test=X[split:]; y_test=Y[split:]

In [106]:
x_train[0].shape

(7, 1)

In [107]:
# LSTM 모델 설계와 학습
model=Sequential()
model.add(LSTM(units=128,activation='relu',input_shape=x_train[0].shape))
model.add(Dense(1))
model.compile(loss='mae',optimizer='adam',metrics=['mae'])
hist=model.fit(x_train,y_train,epochs=200,batch_size=1,validation_data=(x_test,y_test),verbose=2)

Epoch 1/200
250/250 - 2s - loss: 1685.9900 - mae: 1685.9900 - val_loss: 474.6467 - val_mae: 474.6467 - 2s/epoch - 10ms/step
Epoch 2/200
250/250 - 1s - loss: 550.0173 - mae: 550.0173 - val_loss: 337.6149 - val_mae: 337.6149 - 996ms/epoch - 4ms/step
Epoch 3/200
250/250 - 1s - loss: 529.8935 - mae: 529.8935 - val_loss: 493.8566 - val_mae: 493.8566 - 977ms/epoch - 4ms/step
Epoch 4/200
250/250 - 1s - loss: 524.9998 - mae: 524.9998 - val_loss: 322.3449 - val_mae: 322.3449 - 929ms/epoch - 4ms/step
Epoch 5/200
250/250 - 1s - loss: 516.8251 - mae: 516.8251 - val_loss: 566.7067 - val_mae: 566.7067 - 1s/epoch - 4ms/step
Epoch 6/200
250/250 - 1s - loss: 528.4459 - mae: 528.4459 - val_loss: 511.4722 - val_mae: 511.4722 - 1s/epoch - 4ms/step
Epoch 7/200
250/250 - 1s - loss: 561.3167 - mae: 561.3167 - val_loss: 305.9604 - val_mae: 305.9604 - 927ms/epoch - 4ms/step
Epoch 8/200
250/250 - 1s - loss: 491.8047 - mae: 491.8047 - val_loss: 311.7328 - val_mae: 311.7328 - 934ms/epoch - 4ms/step
Epoch 9/200
25

LSTM 모델 평가

In [110]:
ev = model.evaluate(x_test,y_test,verbose = 0)
print(f"손실함수 : {ev[0]}, MAE : {ev[1]}")

손실함수 : 187.8253173828125, MAE : 187.8253173828125


LSTM모델로 예측 수행

In [111]:
pred =  model.predict(x_test)
print(f"평균절댓값 백분율 오차(MAPE) : {sum(abs(y_test-pred)/y_test)/len(x_test)}")

평균절댓값 백분율 오차(MAPE) : [0.02276638]


학습곡선